In [2]:
import pyspch.sp as Sps
import pyspch.display as Spd
import pyspch.core as Spch
import pyspch.nn as Spnn

D:\ProgramFiles\miniconda3\envs\pyspch_edit\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Dummy setup

In [1]:
import random
import torch
import numpy as np

In [33]:
## Dataset

# params
n = 10
label_set = ['a', 'b', 'c']
indim, timedim = 50, 1000
outdim = len(label_set)

# inputs & labels
corpus_list = [str(i) for i in range(n)]
input_list = [np.random.randn(indim, timedim) for _ in range(n)]
label_list = [[random.choice(label_set) for _ in range(timedim)] for _ in range(n)]

# # create relation input/label (mean shift)
# lab2mean = {'a': 20, 'b': -20, 'c': 0} 
# input_list_update = []
# for ins_, labs_ in zip(input_list, label_list):
#     input_list_update.append(np.array([input + lab2mean[lab] for input, lab in zip(ins_, labs_)]))
# input_list = input_list_update

# dataset
spchds = Spnn.SpchDataset(corpus_list, input_list, label_list)

# sampler
lengths = [item.shape[-1] for item in input_list]
spchds.set_sampler(lengths, {'N': 5, 'stride': 1})

# target encoding 
lab2idx = {k: i for i, k in enumerate(label_set)}
spchds.encode_target(lab2idx)

# format
spchds.to_tensor()

# to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
spchds.to_device(device)

In [34]:
# example
x, y = spchds.__getitem__(5)
print(x.shape)
print(y.shape)
print(x.dtype)
print(y.dtype)

torch.Size([450])
torch.Size([])
torch.float32
torch.int64


In [35]:
## Model + training setup

# model
in_dim = x.shape[0]
out_dim = outdim
hidden_dims = [10, 10]
model = Spnn.FFDNN(in_dim, out_dim, hidden_dims)
model.to(device)

# criterion & optimizer
lr = 0.0001
weight_decay = 0
criterion = torch.nn.CrossEntropyLoss() # applies softmax()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# dataloader
batch_size = 64
shuffle = True
train_dl = torch.utils.data.DataLoader(spchds, batch_size=batch_size, shuffle=shuffle)

In [36]:
## Training

# train model
n_epochs = 50
valid_dl = None
patience = 5
every = 5 
current_epoch = 0
tr_loss, val_loss, epoch = Spnn.train(model, train_dl, criterion, optimizer, clip=None, 
    current_epoch=current_epoch, n_epochs=n_epochs,
    valid_dl=valid_dl, patience=patience, every=every)

Epoch 0 -- av. train loss per mini-batch 1.11
Epoch 5 -- av. train loss per mini-batch 1.10
Epoch 10 -- av. train loss per mini-batch 1.10
Epoch 15 -- av. train loss per mini-batch 1.10
Epoch 20 -- av. train loss per mini-batch 1.10
Epoch 25 -- av. train loss per mini-batch 1.10
Epoch 30 -- av. train loss per mini-batch 1.10
Epoch 35 -- av. train loss per mini-batch 1.10
Epoch 40 -- av. train loss per mini-batch 1.10
Epoch 45 -- av. train loss per mini-batch 1.10
